In [ ]:
import json
import random
from typing import Dict, List

# Liste étendue d'ouvertures d'échecs réalistes
OPENINGS = [
    # Défenses Siciliennes
    "Sicilian Defense: Najdorf Variation", "Sicilian Defense: Dragon Variation", 
    "Sicilian Defense: Scheveningen Variation", "Sicilian Defense: Sveshnikov Variation",
    "Sicilian Defense: Accelerated Dragon", "Sicilian Defense: Kan Variation",
    "Sicilian Defense: Taimanov Variation", "Sicilian Defense: Four Knights Variation",
    
    # Ruy Lopez
    "Ruy Lopez: Berlin Defense", "Ruy Lopez: Closed Defense", 
    "Ruy Lopez: Open Defense", "Ruy Lopez: Exchange Variation",
    "Ruy Lopez: Marshall Attack", "Ruy Lopez: Steinitz Defense",
    
    # Défenses Indiennes
    "King's Indian Defense: Classical Variation", "King's Indian Defense: Saemisch Variation",
    "King's Indian Defense: Fianchetto Variation", "Nimzo-Indian Defense: Classical Variation",
    "Nimzo-Indian Defense: Rubinstein Variation", "Queen's Indian Defense: Classical Variation",
    
    # Défenses semi-ouvertes
    "French Defense: Tarrasch Variation", "French Defense: Advance Variation",
    "French Defense: Classical Variation", "Caro-Kann Defense: Classical Variation",
    "Caro-Kann Defense: Advance Variation", "Caro-Kann Defense: Panov-Botvinnik Attack",
    "Pirc Defense: Austrian Attack", "Pirc Defense: Classical Variation",
    "Alekhine's Defense: Modern Variation", "Alekhine's Defense: Four Pawns Attack",
    
    # Gambits
    "Queen's Gambit Declined: Orthodox Defense", "Queen's Gambit Declined: Tartakower Variation",
    "Queen's Gambit Accepted: Classical Variation", "Slav Defense: Semi-Slav",
    "Slav Defense: Exchange Variation", "Benko Gambit: Accepted", "Benko Gambit: Declined",
    "King's Gambit Accepted", "King's Gambit Declined", "Evans Gambit",
    
    # Ouvertures fermées
    "Italian Game: Giuoco Piano", "Italian Game: Two Knights Defense",
    "Italian Game: Giuoco Pianissimo", "Four Knights Game: Spanish Variation",
    "Four Knights Game: Scotch Variation", "Scotch Game: Classical Variation",
    "Vienna Game: Vienna Gambit", "Philidor Defense: Hanham Variation",
    
    # Ouvertures de flanc
    "English Opening: Symmetrical Variation", "English Opening: Reversed Sicilian",
    "Reti Opening: King's Indian Attack", "Bird's Opening: From's Gambit",
    "Nimzo-Larsen Attack: Modern Variation",
    
    # Systèmes
    "London System", "Colle System", "Torre Attack", "Barcza System",
    
    # Ouvertures rares
    "Barnes Opening", "Saragossa Opening", "Amar Opening", "Orangutan Opening",
    "Ware Opening", "Clemenz Opening",
    
    # Défenses rares
    "St. George Defense", "Englund Gambit", "Albin Countergambit", "Latvian Gambit",
    "Elephant Gambit", "Budapest Gambit",
    
    # Variantes modernes
    "Modern Defense: Standard Line", "Pirc-Modern Defense", "Owen's Defense",
    "Lion Defense", "Hippopotamus Defense",
    
    # Gambits divers
    "Smith-Morra Gambit", "Wing Gambit", "Göring Gambit", "Danish Gambit",
]

def generate_opening_stats(base_win_rate_white: float, time_control: str) -> Dict:
    """Génère des statistiques cohérentes selon le contrôle de temps"""
    if time_control == "blitz":
        nb_games = random.randint(20000, 120000)
        popularity = random.uniform(0.15, 0.50)
        accuracy_base = random.uniform(0.70, 0.85)
        win_rate_mult = 1.0
    elif time_control == "rapid":
        nb_games = random.randint(15000, 80000)
        popularity = random.uniform(0.20, 0.45)
        accuracy_base = random.uniform(0.75, 0.90)
        win_rate_mult = 1.05  # Meilleur taux de victoire en rapide
    else:  # bullet
        nb_games = random.randint(30000, 150000)
        popularity = random.uniform(0.10, 0.40)
        accuracy_base = random.uniform(0.65, 0.80)
        win_rate_mult = 0.95  # Plus de déséquilibre en bullet
    
    # Ajustement cohérent du win_rate
    win_rate = round(base_win_rate_white * win_rate_mult, 2)
    win_rate = min(0.65, max(0.35, win_rate))  # Garder dans des limites raisonnables
    
    return {
        "nb_games": nb_games,
        "popularity": round(popularity, 3),
        "win_rate": win_rate,
        "accuracy": round(accuracy_base, 3),
    }

def generate_elo_band_stats(base_win_rate: float, time_control: str) -> Dict[str, Dict]:
    """Génère des statistiques par bande ELO"""
    elo_bands = ["0-500", "500-1000", "1000-1500", "1500-2000", "2000+"]
    stats = {}
    
    # Distribution des parties (plus dans les niveaux moyens)
    total_games_factor = {
        "blitz": 85000,
        "rapid": 45000,
        "bullet": 110000
    }
    
    distribution = [0.05, 0.20, 0.35, 0.25, 0.15]
    
    for i, band in enumerate(elo_bands):
        # Facteur d'ajustement basé sur l'ELO (amélioration avec le niveau)
        elo_factor = i * 0.1  # 0, 0.1, 0.2, 0.3, 0.4
        
        # Win Rate (s'améliore avec l'ELO)
        if base_win_rate > 0.5:
            win_rate_adj = base_win_rate + elo_factor * 0.08
        else:
            win_rate_adj = base_win_rate + elo_factor * 0.06
        
        win_rate_adj = min(0.65, max(0.30, win_rate_adj))
        
        # Accuracy (s'améliore avec l'ELO)
        if time_control == "blitz":
            accuracy_base = 0.60 + elo_factor * 0.08
        elif time_control == "rapid":
            accuracy_base = 0.65 + elo_factor * 0.07
        else:  # bullet
            accuracy_base = 0.55 + elo_factor * 0.08
        
        # Popularité (varie selon le niveau)
        if i == 2:  # 1000-1500 : plus populaire
            popularity = random.uniform(0.30, 0.45)
        elif i == 3:  # 1500-2000
            popularity = random.uniform(0.20, 0.35)
        elif i == 4:  # 2000+
            popularity = random.uniform(0.10, 0.20)
        else:  # Niveaux bas
            popularity = random.uniform(0.02, 0.10)
        
        # Nombre de parties
        nb_games = int(total_games_factor[time_control] * distribution[i] * random.uniform(0.8, 1.2))
        
        stats[band] = {
            "nb_games": max(100, nb_games),
            "win_rate": round(win_rate_adj, 3),
            "popularity": round(popularity, 3),
            "accuracy": round(accuracy_base, 3)
        }
    
    return stats

def generate_opening_data(opening_name: str) -> Dict:
    """Génère les données complètes pour une ouverture"""
    # Déterminer les caractéristiques de base selon le type d'ouverture
    opening_lower = opening_name.lower()
    
    if "sicilian" in opening_lower:
        # Les Siciliennes sont favorables aux Noirs
        white_win_rate = random.uniform(0.38, 0.45)
        black_win_rate = random.uniform(0.40, 0.50)
    elif "ruy lopez" in opening_lower or "italian" in opening_lower:
        # Les ouvertures espagnoles sont favorables aux Blancs
        white_win_rate = random.uniform(0.48, 0.55)
        black_win_rate = random.uniform(0.30, 0.38)
    elif "queen's gambit" in opening_lower or "slav" in opening_lower:
        # Défenses solides
        white_win_rate = random.uniform(0.46, 0.52)
        black_win_rate = random.uniform(0.32, 0.38)
    elif "french" in opening_lower or "caro-kann" in opening_lower:
        # Défenses fermées
        white_win_rate = random.uniform(0.44, 0.50)
        black_win_rate = random.uniform(0.34, 0.40)
    elif "king's indian" in opening_lower or "nimzo-indian" in opening_lower:
        # Défenses indiennes
        white_win_rate = random.uniform(0.46, 0.52)
        black_win_rate = random.uniform(0.35, 0.42)
    elif "gambit" in opening_lower:
        # Les gambits mènent à des positions dynamiques
        white_win_rate = random.uniform(0.42, 0.55)
        black_win_rate = random.uniform(0.35, 0.45)
    else:
        # Valeurs par défaut
        white_win_rate = random.uniform(0.45, 0.52)
        black_win_rate = random.uniform(0.32, 0.40)
    
    # S'assurer que les valeurs sont cohérentes
    white_win_rate = round(white_win_rate, 3)
    black_win_rate = round(black_win_rate, 3)
    draw_rate = round(1 - white_win_rate - black_win_rate, 3)
    
    # Ajustement final pour que la somme fasse exactement 1
    total = white_win_rate + black_win_rate + draw_rate
    if total != 1.0:
        diff = 1.0 - total
        draw_rate += diff
        draw_rate = round(draw_rate, 3)
    
    # Générer les statistiques par contrôle de temps
    opening_explorer = {}
    for time_control in ["blitz", "rapid", "bullet"]:
        base_stats = generate_opening_stats(white_win_rate, time_control)
        base_stats["elo_bands"] = generate_elo_band_stats(
            base_stats["win_rate"], 
            time_control
        )
        opening_explorer[time_control] = base_stats
    
    return {
        "name": opening_name,
        "white_win_rate": white_win_rate,
        "black_win_rate": black_win_rate,
        "draw_rate": draw_rate,
        "opening-explorer": opening_explorer
    }

def generate_unique_openings(count: int = 200) -> List[Dict]:
    """Génère une liste d'ouvertures uniques"""
    data_list = []
    used_names = set()
    
    # Étendre la liste d'ouvertures si nécessaire
    all_openings = OPENINGS.copy()
    
    # Ajouter des variantes numérotées pour atteindre le compte désiré
    i = 0
    while len(data_list) < count:
        if i < len(all_openings):
            base_name = all_openings[i]
        else:
            # Si on a épuisé la liste, utiliser un nom générique
            base_name = f"Chess Opening Variation {i+1}"
        
        # Ajouter des variations pour rendre les noms uniques
        variations = ["Main Line", "Classical Variation", "Modern Variation", 
                     "Attack", "Defense", "Gambit", "System", "Line"]
        
        if i < len(all_openings):
            opening_name = base_name
        else:
            variation = random.choice(variations)
            opening_name = f"{base_name} {variation}"
        
        # Ajouter un numéro si nécessaire pour éviter les doublons
        if opening_name in used_names:
            opening_name = f"{opening_name} {len(used_names) + 1}"
        
        used_names.add(opening_name)
        data_list.append(generate_opening_data(opening_name))
        i += 1
    
    return data_list

def main():
    # Générer 200 ouvertures
    print("Génération de 200 ouvertures d'échecs...")
    chess_data = generate_unique_openings(200)
    
    # Sauvegarder dans un fichier JSON
    with open('200_chess_openings_realistic.json', 'w', encoding='utf-8') as f:
        json.dump(chess_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Fichier '200_chess_openings_realistic.json' créé avec succès!")
    print(f"📊 {len(chess_data)} ouvertures générées")
    
    # Afficher un aperçu
    print("\n📋 Aperçu des premières ouvertures:")
    for i in range(min(3, len(chess_data))):
        opening = chess_data[i]
        print(f"\n{i+1}. {opening['name']}")
        print(f"   Blancs: {opening['white_win_rate']*100:.1f}% | "
              f"Noirs: {opening['black_win_rate']*100:.1f}% | "
              f"Nuls: {opening['draw_rate']*100:.1f}%")
    
    # Statistiques globales
    total_games_blitz = sum(op['opening-explorer']['blitz']['nb_games'] for op in chess_data)
    total_games_rapid = sum(op['opening-explorer']['rapid']['nb_games'] for op in chess_data)
    total_games_bullet = sum(op['opening-explorer']['bullet']['nb_games'] for op in chess_data)
    
    print(f"\n📈 Statistiques globales:")
    print(f"   Total parties blitz: {total_games_blitz:,}")
    print(f"   Total parties rapid: {total_games_rapid:,}")
    print(f"   Total parties bullet: {total_games_bullet:,}")
    print(f"   Total parties toutes catégories: {total_games_blitz + total_games_rapid + total_games_bullet:,}")

if __name__ == "__main__":
    main()

Génération de 200 ouvertures d'échecs...
✅ Fichier '200_chess_openings_realistic.json' créé avec succès!
📊 200 ouvertures générées

📋 Aperçu des premières ouvertures:

1. Sicilian Defense: Najdorf Variation
   Blancs: 38.5% | Noirs: 44.2% | Nuls: 17.3%

2. Sicilian Defense: Dragon Variation
   Blancs: 42.0% | Noirs: 40.1% | Nuls: 17.9%

3. Sicilian Defense: Scheveningen Variation
   Blancs: 39.3% | Noirs: 46.5% | Nuls: 14.2%

📈 Statistiques globales:
   Total parties blitz: 14,088,474
   Total parties rapid: 9,432,039
   Total parties bullet: 17,129,697
   Total parties toutes catégories: 40,650,210
